<a href="https://colab.research.google.com/github/ShubhamMohanty680/InceptionV3/blob/main/cat_dog_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# download the data from g drive

import gdown
url = "https://drive.google.com/file/d/16iH0_n2UdcdfgOv4IbKEUY8dKYvNsKrh/view?usp=sharing"
file_id = url.split("/")[-2]
print(file_id)
prefix = 'https://drive.google.com/uc?/export=download&id='
gdown.download(prefix+file_id, "catdog.zip")

16iH0_n2UdcdfgOv4IbKEUY8dKYvNsKrh


Downloading...
From: https://drive.google.com/uc?/export=download&id=16iH0_n2UdcdfgOv4IbKEUY8dKYvNsKrh
To: /content/catdog.zip
100%|██████████| 9.09M/9.09M [00:00<00:00, 28.4MB/s]


'catdog.zip'

In [3]:
!unzip catdog.zip

Archive:  catdog.zip
   creating: train/
   creating: train/Cat/
  inflating: train/Cat/0.jpg         
  inflating: train/Cat/1.jpg         
  inflating: train/Cat/2.jpg         
  inflating: train/Cat/cat.2405.jpg  
  inflating: train/Cat/cat.2406.jpg  
  inflating: train/Cat/cat.2436.jpg  
  inflating: train/Cat/cat.2437.jpg  
  inflating: train/Cat/cat.2438.jpg  
  inflating: train/Cat/cat.2439.jpg  
  inflating: train/Cat/cat.2440.jpg  
  inflating: train/Cat/cat.2441.jpg  
  inflating: train/Cat/cat.2442.jpg  
  inflating: train/Cat/cat.2443.jpg  
  inflating: train/Cat/cat.2444.jpg  
  inflating: train/Cat/cat.2445.jpg  
  inflating: train/Cat/cat.2446.jpg  
  inflating: train/Cat/cat.2447.jpg  
  inflating: train/Cat/cat.2448.jpg  
  inflating: train/Cat/cat.2449.jpg  
  inflating: train/Cat/cat.2450.jpg  
  inflating: train/Cat/cat.2451.jpg  
  inflating: train/Cat/cat.2452.jpg  
  inflating: train/Cat/cat.2453.jpg  
  inflating: train/Cat/cat.2454.jpg  
  inflating: train/Cat/

### Training

In [6]:
from tensorflow import keras
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten

In [7]:
# Set the path to your training and validation data
train_data_dir = '/content/train'
validation_data_dir = '/content/validation'

In [8]:
# Set the number of training and validation samples
num_train_samples = 337
num_validation_samples = 59


In [9]:
# Set the number of epochs and batch size
epochs = 5
batch_size = 16

In [10]:
# Load the InceptionV3 model without the top layer
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

87910968/87910968 [==============================] - 0s 0us/step


In [11]:
# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

In [12]:
# Create a new model
model = Sequential()

# Add the base model as a layer
model.add(base_model)

# Add custom layers on top of the base model
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 flatten (Flatten)           (None, 51200)             0         
                                                                 
 dense (Dense)               (None, 256)               13107456  
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 34910497 (133.17 MB)
Trainable params: 13107713 (50.00 MB)
Non-trainable params: 21802784 (83.17 MB)
_________________________________________________________________


In [14]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [15]:
# Preprocess the training and validation data
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [16]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary')

Found 337 images belonging to 2 classes.
Found 59 images belonging to 2 classes.


In [17]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=num_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=num_validation_samples // batch_size)

Epoch 1/5
21/21 [==============================] - 35s 1s/step - loss: 3.5805 - accuracy: 0.8692 - val_loss: 1.1902 - val_accuracy: 0.9583
Epoch 2/5
21/21 [==============================] - 31s 2s/step - loss: 1.1183 - accuracy: 0.9782 - val_loss: 1.1820 - val_accuracy: 0.9583
Epoch 3/5
21/21 [==============================] - 30s 1s/step - loss: 0.2715 - accuracy: 0.9875 - val_loss: 4.3293 - val_accuracy: 0.9375
Epoch 4/5
21/21 [==============================] - 32s 2s/step - loss: 0.0592 - accuracy: 0.9844 - val_loss: 1.0925 - val_accuracy: 0.9583
Epoch 5/5
21/21 [==============================] - 33s 2s/step - loss: 0.1741 - accuracy: 0.9875 - val_loss: 2.8109 - val_accuracy: 0.9375


In [18]:
# Save the trained model
model.save('dog_cat_classifier.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [20]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import os

In [21]:
model=load_model('/content/dog_cat_classifier.h5')

In [23]:
img_path='/content/train/Cat/1.jpg'
test_image=image.load_img(img_path,target_size=(224,224))
test_image=image.img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)
result=np.argmax(model.predict(test_image),axis=1)
print(result)

1/1 [==============================] - 2s 2s/step
[0]


In [24]:
if result[0]==1:
  prediction='dog'
  print(prediction)
else:
  prediction='cat'
  print(prediction)

cat
